In [1]:
import requests
import time
import datetime
import pandas as pd

import hashlib
import hmac
import base64

import json

from pprint import pprint 

from pandas.io.json import json_normalize

import numpy

In [2]:
# Current time stamp
timestamp = int(time.time())

In [3]:
# Date
today_date = datetime.date.today() + datetime.timedelta(days=0)
yesterday_date = datetime.date.today() + datetime.timedelta(days=-1)
seven_days_before_date = datetime.date.today() + datetime.timedelta(days=-7)

In [4]:
# Shop Parameter
partner_id = 12156
shopid = 25482220
shop_key = '60dbb2b5bcd937be4d6101cf70677908da72f4988c533d0c07b997fd22acd4f2'

In [5]:
# get_auth_signature
def get_auth_signature(url, payload):
    # API Key
    shopee_secret = shop_key
    # Post Message
    post_message = url + '|' + json.dumps(payload)
    # HMAC-SHA256
    message = bytes(post_message, 'utf-8')
    secret = bytes(shopee_secret, 'utf-8')
    hash = hmac.new(secret, message, hashlib.sha256)
    auth_signature = hash.hexdigest()
    return auth_signature

In [6]:
# get item list URL
url = 'https://partner.shopeemobile.com/api/v1/items/get'

In [10]:
# get all item
max_page = 20
pagination_entries_per_page = 100
i = 0

try:
    while i <= max_page:
        # start entry
        start_entry = i * pagination_entries_per_page

        # Parameter
        pagination_offset = start_entry

        # payload
        payload = {"partner_id":partner_id,
                   "shopid":shopid,
                   "timestamp":timestamp,
                   "pagination_offset":pagination_offset,
                   "pagination_entries_per_page":pagination_entries_per_page}
        # Auth Signature
        Auth_signature = get_auth_signature(url, payload)
        # HTML Header
        headers = {"Host": "partner.shopeemobile.com",
                    "Content-Type": "application/json",
                    "Content-Length": "89",
                    "Authorization": Auth_signature}
        # HTML Body
        payload = json.dumps(payload)
        # Post Request
        r = requests.post(url, data=payload, headers=headers)
        # 打印原始Json
        parsed = json.loads(r.text)
        # 漂亮打印Json
        item_list = json_normalize(parsed['items'])
        # 排除update_time
        selected_column = ['item_id', 'shopid', 'status']
        item_list = item_list[selected_column]

        # 合并
        if i == 0:
            all_item_list = item_list
        else:
            all_item_list = all_item_list.append(item_list)

        i += 1
        print('now i is ' + str(i))
except:
    pass

# 去重
all_item_list = all_item_list.drop_duplicates('item_id').reset_index()
all_item_list

now i is 1
now i is 2
now i is 3
now i is 4
now i is 5
now i is 6
now i is 7
now i is 8
now i is 9
now i is 10
now i is 11
now i is 12


,index,item_id,shopid,status
0,0,712457456,25482220,NORMAL
1,1,812561258,25482220,NORMAL
2,2,799036297,25482220,NORMAL
3,3,827981212,25482220,NORMAL
4,4,848072934,25482220,NORMAL
5,5,845114948,25482220,NORMAL
6,6,828277747,25482220,NORMAL
7,7,848013389,25482220,NORMAL
8,8,865749062,25482220,NORMAL
9,9,865762957,25482220,NORMAL


In [12]:
# 更新商品信息
item_update_url = 'https://partner.shopeemobile.com/api/v1/item/update'

In [13]:
# 参数
days_to_ship = 5

In [16]:
# 历遍
for item_id in all_item_list['item_id']:

    timestamp = int(time.time())

    payload = {"partner_id":partner_id,
               "shopid":shopid,
               "timestamp":timestamp,
               "item_id":int(item_id),
               "days_to_ship":days_to_ship}

    # print(json.dumps(payload))

    Auth_signature = get_auth_signature(item_update_url, payload)
    # HTML Header

    headers = {"Host": "partner.shopeemobile.com",
                "Content-Type": "application/json",
                "Content-Length": "89",
                "Authorization": Auth_signature}
    # HTML Body
    payload = json.dumps(payload)
    # Post Request
    r = None
    try_time = 0

    while r is None:
        try:
            r = requests.post(item_update_url, data=payload, headers=headers)
        except Exception as err:
            try_time += 1
            print('An exception happened: ' + str(err) + ", post fails. Try again. Try " + str(try_time) + " time.")
            pass

    print(r.text)
    # Notice
    print(str(item_id) + ' days_to_ship is updated.')
    time.sleep(50)

{"msg":"The Days to Ship should be bigger than 7.","error":"error_param"}
712457456 days_to_ship is updated.


KeyboardInterrupt: 